# NAVERMAP 크롤링

분석 과정에서 필요한 정보(업종, 가격, 태그 등)의 데이터를 보완하기 위해 F&B 관광지 쇼핑 데이터 네이버맵 검색 크롤링


## 1. F&B 데이터 크롤링

In [ ]:
import time
from tqdm import tqdm
import pandas as pd
from selenium import webdriver
import warnings

warnings.filterwarnings('ignore')

In [213]:
base_url = "https://map.naver.com/v5/?c=14063101.9902798,3955242.8346131,13,0,0,0,dh&entry=plt"

In [214]:
driver = webdriver.Chrome(executable_path = 'D:\Python_417\chromedriver')
driver.get(url=base_url)

In [58]:
food = pd.read_csv('data/F&B데이터_co2.csv', encoding='utf-8-sig')

In [138]:
# ------------------- 검색창에 검색어 입력
def search_box(driver, title):
    search_box = driver.find_element_by_css_selector('div.input_box > input')
    search_box.clear()
    search_box.send_keys('제주도'+ ' ' + title)
    search_box.send_keys('\n')

In [161]:
def naver_contents(name_list, type_list, address_list, visitor_review_num_list, blog_review_num_list, visitor_review_list, menu_list, dl_mood_list, dl_topic_list, dl_why_list, dl_20_pop_list, already_crawling_list):
    name, type, address, visitor_review_num, blog_review_num, visitor_review, menu, dl_mood, dl_topic, dl_why, dl_20_pop = ['']*11

    # TITLE
    name = driver.find_element_by_css_selector('span.Fc1rA').text
    if name not in already_crawling_list:
        # TYPE
        type = driver.find_element_by_css_selector('span.DJJvD').text
        # ADDRESS
        try:
            address = driver.find_element_by_css_selector('span.IH7VW').text
        except:
            driver.find_element_by_css_selector('div.ngGKH > div > div > a.tpj9w._tab-menu').click()
            time.sleep(1)
            driver.find_element_by_css_selector('div.ngGKH > div > div > a.tpj9w._tab-menu').click()
            time.sleep(1)
            address = driver.find_element_by_css_selector('span.IH7VW').text

        # 리뷰 num
        if len(driver.find_elements_by_css_selector('span.PXMot')) >= 2 :
            if '블로그리뷰' in driver.find_elements_by_css_selector('span.PXMot')[-1].text and '방문자리뷰' in driver.find_elements_by_css_selector('span.PXMot')[-2].text :
                visitor_review_num = int(driver.find_elements_by_css_selector('span.PXMot')[-2].text.split(' ')[1].replace(',',''))
                blog_review_num = int(driver.find_elements_by_css_selector('span.PXMot')[-1].text.split(' ')[1].replace(',',''))
            elif '블로그리뷰' in driver.find_elements_by_css_selector('span.PXMot')[-1].text :
                visitor_review_num = 0
                blog_review_num = int(driver.find_elements_by_css_selector('span.PXMot')[-1].text.split(' ')[1].replace(',',''))
            elif '방문자리뷰' in driver.find_elements_by_css_selector('span.PXMot')[-1].text :
                visitor_review_num = int(driver.find_elements_by_css_selector('span.PXMot')[-1].text.split(' ')[1].replace(',',''))
                blog_review_num = 0
        else:
            try:
                if driver.find_element_by_css_selector('span.PXMot').text.split(' ')[0] == '방문자리뷰':
                    time.sleep(1)
                    visitor_review_num = int(driver.find_element_by_css_selector('span.PxMot').text.split(' ')[1].replace(',',''))
                    blog_review_num = 0
                else:
                    visitor_review_num = 0
                    blog_review_num = int(driver.find_element_by_css_selector('span.PXMot').text.split(' ')[1].replace(',',''))
            except:
                visitor_review_num, blog_review_num = 0,0

        try:
            menu = driver.find_elements_by_css_selector('div.gl2cc')
            menu = [ int(menu[idx].text.replace(',','').replace('원','')) for idx in range(len(menu)) ]
            menu = int(sum(menu)/len(menu))
        except:
            try:
                menu = driver.find_elements_by_css_selector('em.awlpp')
                menu = [ int(menu[idx].text.replace(',','').replace('원','')) for idx in range(len(menu)) ]
                menu = int(sum(menu)/len(menu))
            except:
                menu = '알 수 없음'

        try:
            # DATA LAB
            driver.find_element_by_css_selector('div.place_section.I_y6k > div.lfH3O > a').click()
            time.sleep(1)
            dl = driver.find_elements_by_css_selector('li.nc5wr')
            for dl in dl:
                if '분위기' in dl.text:
                    dl_mood = dl.text[3:]
                elif '인기토픽' in dl.text:
                    dl_topic = dl.text[4:]
                elif '찾는목적' in dl.text:
                    dl_why = dl.text[4:]
        except:
            pass
        try:
            dl_20_pop = driver.find_elements_by_css_selector('li.JkrLe')[1].text.split('\n')[0]
        except:
            pass
        # 방문자 리뷰
        if visitor_review_num > 10:
            try:
                visitor_review = driver.find_element_by_css_selector('ul.uNsI9').text.split('\n')
                visitor_review = { visitor_review[idx][1:-1] : int(visitor_review[idx+2])  for idx in range(0,len(visitor_review),3) }
            except:
                try:
                    driver.execute_script('window.scrollTo(0, 3000);')
                    driver.implicitly_wait(5)
                    visitor_review = driver.find_element_by_css_selector('ul.uNsI9').text.split('\n')
                    visitor_review = { visitor_review[idx][1:-1] : int(visitor_review[idx+2])  for idx in range(0,len(visitor_review),3) }
                except:
                    try:
                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
                        time.sleep(2)
                        visitor_review = driver.find_element_by_css_selector('ul.uNsI9').text.split('\n')
                        visitor_review = { visitor_review[idx][1:-1] : int(visitor_review[idx+2])  for idx in range(0,len(visitor_review),3) }
                    except:
                        visitor_review = 'no_result'
    else:
        pass

    name_list.append(name)
    type_list.append(type)
    address_list.append(address)
    visitor_review_num_list.append(visitor_review_num)
    blog_review_num_list.append(blog_review_num)
    visitor_review_list.append(visitor_review)
    menu_list.append(menu)
    dl_mood_list.append(dl_mood)
    dl_topic_list.append(dl_topic)
    dl_why_list.append(dl_why)
    dl_20_pop_list.append(dl_20_pop)

    return name_list, type_list, address_list, visitor_review_num_list, blog_review_num_list, visitor_review_list, menu_list, dl_mood_list, dl_topic_list, dl_why_list, dl_20_pop_list

In [186]:
name_list, type_list, address_list, visitor_review_num_list, blog_review_num_list, visitor_review_list, menu_list, dl_mood_list, dl_topic_list, dl_why_list, dl_20_pop_list = [], [], [], [], [], [], [], [], [], [], []

In [163]:
org_list = []
already_crawling_list= []

In [211]:
already_crawling_list = org_list.copy()
len(already_crawling_list)

633

In [216]:
column_lists = [name_list, type_list, address_list, visitor_review_num_list, blog_review_num_list, visitor_review_list, menu_list, dl_mood_list, dl_topic_list, dl_why_list, dl_20_pop_list]
# ---------------------------------------------------
# ---------------------------------------------------
#-> 처음 돌릴 땐 아래 코드도 실행해줘야 함
for title in crawling_list[0]:
    try:
        driver.switch_to.frame('entryIframe')
    except:
        pass

for title in tqdm(crawling_list):

    no_result = False
    if title in already_crawling_list:
        pass
    else:
        driver.switch_to.default_content() # Iframe 빠져나가기
        search_box = driver.find_element_by_css_selector('div.input_box > input')
        search_box.clear()
        search_box.send_keys('제주도' + ' ' + title)
        search_box.send_keys('\n')
        time.sleep(2)

        try:
            # 바로 나온 경우
            driver.switch_to.frame('entryIframe') # Iframe으로 들어가야 함
        except:
            # 바로 나오지 않고 목록에서 선택 해야 하는 경우
            driver.switch_to.frame('searchIframe') # Iframe으로 들어가야 함
            try:
                tmp = driver.find_element_by_css_selector('div.FYvSc').text
                no_result = True
            except:
                try:
                    driver.find_element_by_css_selector('li.VLTHu.OW9LQ > div.ouxiq.icT4K > a').click() # 맨 위에 있는 거 click
                except:
                    try:
                        driver.find_element_by_css_selector('li.UEzoS.rTjJo > div > a').click()
                    except:
                        try:
                            driver.find_element_by_css_selector('li.VLTHu.OW9LQ > div.ouxiq > a').cilck()
                        except:
                            driver.find_element_by_css_selector('li.VLTHu.OW9LQ > div.ouxiq > a').click()
                driver.switch_to.default_content() # Iframe 빠져나가기
                time.sleep(1)
                driver.switch_to.frame('entryIframe') # Iframe으로 들어가야 함

        time.sleep(2)
        if no_result:
            for col in column_lists:
                col.append('no_result')
        else:
            name_list, type_list, address_list, visitor_review_num_list, blog_review_num_list, visitor_review_list, menu_list, dl_mood_list, dl_topic_list, dl_why_list, dl_20_pop_list = naver_contents(name_list, type_list, address_list, visitor_review_num_list, blog_review_num_list, visitor_review_list, menu_list, dl_mood_list, dl_topic_list, dl_why_list, dl_20_pop_list, already_crawling_list)
        org_list.append(title)
    print(name_list[-1], visitor_review_num_list[-1], visitor_review_list[-1], dl_20_pop_list[-1])
    driver.switch_to.default_content() # Iframe 빠져나가기

  2%|▏         | 25/1590 [00:00<00:06, 249.17it/s]

블리케이크 71 {'디저트가 맛있어요': 17, '특별한 메뉴가 있어요': 9, '친절해요': 7, '매장이 청결해요': 5, '커피가 맛있어요': 3} 
블리케이크 71 {'디저트가 맛있어요': 17, '특별한 메뉴가 있어요': 9, '친절해요': 7, '매장이 청결해요': 5, '커피가 맛있어요': 3} 
블리케이크 71 {'디저트가 맛있어요': 17, '특별한 메뉴가 있어요': 9, '친절해요': 7, '매장이 청결해요': 5, '커피가 맛있어요': 3} 
블리케이크 71 {'디저트가 맛있어요': 17, '특별한 메뉴가 있어요': 9, '친절해요': 7, '매장이 청결해요': 5, '커피가 맛있어요': 3} 
블리케이크 71 {'디저트가 맛있어요': 17, '특별한 메뉴가 있어요': 9, '친절해요': 7, '매장이 청결해요': 5, '커피가 맛있어요': 3} 
블리케이크 71 {'디저트가 맛있어요': 17, '특별한 메뉴가 있어요': 9, '친절해요': 7, '매장이 청결해요': 5, '커피가 맛있어요': 3} 
블리케이크 71 {'디저트가 맛있어요': 17, '특별한 메뉴가 있어요': 9, '친절해요': 7, '매장이 청결해요': 5, '커피가 맛있어요': 3} 
블리케이크 71 {'디저트가 맛있어요': 17, '특별한 메뉴가 있어요': 9, '친절해요': 7, '매장이 청결해요': 5, '커피가 맛있어요': 3} 
블리케이크 71 {'디저트가 맛있어요': 17, '특별한 메뉴가 있어요': 9, '친절해요': 7, '매장이 청결해요': 5, '커피가 맛있어요': 3} 
블리케이크 71 {'디저트가 맛있어요': 17, '특별한 메뉴가 있어요': 9, '친절해요': 7, '매장이 청결해요': 5, '커피가 맛있어요': 3} 
블리케이크 71 {'디저트가 맛있어요': 17, '특별한 메뉴가 있어요': 9, '친절해요': 7, '매장이 청결해요': 5, '커피가 맛있어요': 3} 
블리케이크 71 {'디저트가 맛있어요': 17, '특별한 메뉴가 있어요': 9

  5%|▌         | 83/1590 [00:00<00:05, 281.63it/s]

블리케이크 71 {'디저트가 맛있어요': 17, '특별한 메뉴가 있어요': 9, '친절해요': 7, '매장이 청결해요': 5, '커피가 맛있어요': 3} 
블리케이크 71 {'디저트가 맛있어요': 17, '특별한 메뉴가 있어요': 9, '친절해요': 7, '매장이 청결해요': 5, '커피가 맛있어요': 3} 
블리케이크 71 {'디저트가 맛있어요': 17, '특별한 메뉴가 있어요': 9, '친절해요': 7, '매장이 청결해요': 5, '커피가 맛있어요': 3} 
블리케이크 71 {'디저트가 맛있어요': 17, '특별한 메뉴가 있어요': 9, '친절해요': 7, '매장이 청결해요': 5, '커피가 맛있어요': 3} 
블리케이크 71 {'디저트가 맛있어요': 17, '특별한 메뉴가 있어요': 9, '친절해요': 7, '매장이 청결해요': 5, '커피가 맛있어요': 3} 
블리케이크 71 {'디저트가 맛있어요': 17, '특별한 메뉴가 있어요': 9, '친절해요': 7, '매장이 청결해요': 5, '커피가 맛있어요': 3} 
블리케이크 71 {'디저트가 맛있어요': 17, '특별한 메뉴가 있어요': 9, '친절해요': 7, '매장이 청결해요': 5, '커피가 맛있어요': 3} 
블리케이크 71 {'디저트가 맛있어요': 17, '특별한 메뉴가 있어요': 9, '친절해요': 7, '매장이 청결해요': 5, '커피가 맛있어요': 3} 
블리케이크 71 {'디저트가 맛있어요': 17, '특별한 메뉴가 있어요': 9, '친절해요': 7, '매장이 청결해요': 5, '커피가 맛있어요': 3} 
블리케이크 71 {'디저트가 맛있어요': 17, '특별한 메뉴가 있어요': 9, '친절해요': 7, '매장이 청결해요': 5, '커피가 맛있어요': 3} 
블리케이크 71 {'디저트가 맛있어요': 17, '특별한 메뉴가 있어요': 9, '친절해요': 7, '매장이 청결해요': 5, '커피가 맛있어요': 3} 
블리케이크 71 {'디저트가 맛있어요': 17, '특별한 메뉴가 있어요': 9

  9%|▉         | 142/1590 [00:00<00:05, 288.63it/s]

...

흑소랑 204 {'음식이 맛있어요': 52, '친절해요': 28, '재료가 신선해요': 22, '주차하기 편해요': 19, '매장이 청결해요': 16} 77.47249999999998%


100%|█████████▉| 1589/1590 [7:04:08<00:22, 22.14s/it]

흑염소마루 남원본점 144 {'음식이 맛있어요': 33, '특별한 메뉴가 있어요': 13, '주차하기 편해요': 10, '재료가 신선해요': 9, '친절해요': 6} 


100%|██████████| 1590/1590 [7:04:25<00:00, 16.02s/it]

흑한우명품관 364 {'음식이 맛있어요': 98, '주차하기 편해요': 55, '매장이 넓어요': 33, '재료가 신선해요': 28, '매장이 청결해요': 25} 69.49999999999999%


In [219]:
data_tmp = pd.DataFrame([name_list, type_list, address_list, visitor_review_num_list, blog_review_num_list, visitor_review_list, menu_list, dl_mood_list, dl_topic_list, dl_why_list, dl_20_pop_list, org_list]).T
data_tmp.columns = ['name_list', 'type_list', 'address_list', 'visitor_review_num_list', 'blog_review_num_list', 'visitor_review_list', 'menu_list', 'dl_mood_list', 'dl_topic_list', 'dl_why_list', 'dl_20_pop_list', 'org_list']

In [220]:
data_tmp.to_csv('data/F&B데이터_네이버크롤링.csv', encoding='utf-8-sig', index=False)

## 2. 쇼핑 데이터 크롤링


In [8]:
import time
from tqdm import tqdm
import pandas as pd
from selenium import webdriver
import warnings
import os

In [51]:
warnings.filterwarnings('ignore')

In [53]:
shopping = pd.read_csv('data/쇼핑데이터_co2.csv', encoding='utf-8-sig')

In [54]:
shopping['AREA_NM_PRE']

0        b일상잡화점
1        han컬렉션
2           갈중이
3            게른
4          곱은과일
         ...   
117      한마음한라봉
118         한복담
119    함덕민속오일시장
120      현승이네농장
121      화북종합시장
Name: AREA_NM_PRE, Length: 122, dtype: object

In [55]:
base_url = "https://map.naver.com/v5/?c=14063101.9902798,3955242.8346131,13,0,0,0,dh&entry=plt"

In [57]:
path= "C://Users//hjh05//Desktop//mefasol//code//chromedriver.exe"
driver = webdriver.Chrome(path)

In [58]:
driver.get(url=base_url)

In [59]:
# ------------------- 검색창에 검색어 입력
def search_box(driver, title):
    search_box = driver.find_element_by_css_selector('div.input_box > input')
    search_box.clear()
    search_box.send_keys('제주도'+ ' ' + title)
    search_box.send_keys('\n')

In [60]:
#driver.find_element_by_css_selector('div.place_section.I_y6k > div.lfH3O').click()

In [61]:
def naver_contents(name_list, type_list, address_list, visitor_review_num_list, blog_review_num_list, visitor_review_list, dl_mood_list, dl_topic_list, dl_why_list, dl_20_pop_list, already_crawling_list):
    name, type, address, visitor_review_num, blog_review_num, visitor_review, dl_mood, dl_topic, dl_why, dl_20_pop = ['']*10

    # TITLE
    name = driver.find_element_by_css_selector('span.Fc1rA').text
    if name not in already_crawling_list:
        # TYPE
        type = driver.find_element_by_css_selector('span.DJJvD').text
        # ADDRESS
        try:
            address = driver.find_element_by_css_selector('span.IH7VW').text
        except:
            driver.find_element_by_css_selector('div.ngGKH > div > div > a.tpj9w._tab-menu').click()
            time.sleep(1)
            driver.find_element_by_css_selector('div.ngGKH > div > div > a.tpj9w._tab-menu').click()
            time.sleep(1)
            address = driver.find_element_by_css_selector('span.IH7VW').text

        # 리뷰 num
        if len(driver.find_elements_by_css_selector('span.PXMot')) >= 2 :
            if '블로그리뷰' in driver.find_elements_by_css_selector('span.PXMot')[-1].text and '방문자리뷰' in driver.find_elements_by_css_selector('span.PXMot')[-2].text :
                visitor_review_num = int(driver.find_elements_by_css_selector('span.PXMot')[-2].text.split(' ')[1].replace(',',''))
                blog_review_num = int(driver.find_elements_by_css_selector('span.PXMot')[-1].text.split(' ')[1].replace(',',''))
            elif '블로그리뷰' in driver.find_elements_by_css_selector('span.PXMot')[-1].text :
                visitor_review_num = 0
                blog_review_num = int(driver.find_elements_by_css_selector('span.PXMot')[-1].text.split(' ')[1].replace(',',''))
            elif '방문자리뷰' in driver.find_elements_by_css_selector('span.PXMot')[-1].text :
                visitor_review_num = int(driver.find_elements_by_css_selector('span.PXMot')[-1].text.split(' ')[1].replace(',',''))
                blog_review_num = 0
        else:
            try:
                if driver.find_element_by_css_selector('span.PXMot').text.split(' ')[0] == '방문자리뷰':
                    time.sleep(1)
                    visitor_review_num = int(driver.find_element_by_css_selector('span.PxMot').text.split(' ')[1].replace(',',''))
                    blog_review_num = 0
                else:
                    visitor_review_num = 0
                    blog_review_num = int(driver.find_element_by_css_selector('span.PXMot').text.split(' ')[1].replace(',',''))
            except:
                visitor_review_num, blog_review_num = 0,0

        try:
            # DATA LAB
            driver.find_element_by_css_selector('div.place_section.I_y6k > div.lfH3O > a').click()
            time.sleep(1)
            dl = driver.find_elements_by_css_selector('li.nc5wr')
            for dl in dl:
                if '분위기' in dl.text:
                    dl_mood = dl.text[3:]
                elif '인기토픽' in dl.text:
                    dl_topic = dl.text[4:]
                elif '찾는목적' in dl.text:
                    dl_why = dl.text[4:]
        except:
            pass
        try:
            dl_20_pop = driver.find_elements_by_css_selector('li.JkrLe')[1].text.split('\n')[0]
        except:
            pass
        # 방문자 리뷰
        if visitor_review_num > 10:
            try:
                visitor_review = driver.find_element_by_css_selector('ul.uNsI9').text.split('\n')
                visitor_review = { visitor_review[idx][1:-1] : int(visitor_review[idx+2])  for idx in range(0,len(visitor_review),3) }
            except:
                try:
                    driver.execute_script('window.scrollTo(0, 3000);')
                    driver.implicitly_wait(5)
                    visitor_review = driver.find_element_by_css_selector('ul.uNsI9').text.split('\n')
                    visitor_review = { visitor_review[idx][1:-1] : int(visitor_review[idx+2])  for idx in range(0,len(visitor_review),3) }
                except:
                    try:
                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
                        time.sleep(2)
                        visitor_review = driver.find_element_by_css_selector('ul.uNsI9').text.split('\n')
                        visitor_review = { visitor_review[idx][1:-1] : int(visitor_review[idx+2])  for idx in range(0,len(visitor_review),3) }
                    except:
                        visitor_review = 'no_result'
    else:
        pass

    name_list.append(name)
    type_list.append(type)
    address_list.append(address)
    visitor_review_num_list.append(visitor_review_num)
    blog_review_num_list.append(blog_review_num)
    visitor_review_list.append(visitor_review)
    dl_mood_list.append(dl_mood)
    dl_topic_list.append(dl_topic)
    dl_why_list.append(dl_why)
    dl_20_pop_list.append(dl_20_pop)

    return name_list, type_list, address_list, visitor_review_num_list, blog_review_num_list, visitor_review_list, dl_mood_list, dl_topic_list, dl_why_list, dl_20_pop_list

In [62]:
name_list, type_list, address_list, visitor_review_num_list, blog_review_num_list, visitor_review_list, dl_mood_list, dl_topic_list, dl_why_list, dl_20_pop_list = [], [], [], [], [], [], [], [], [], []

In [63]:
org_list = []
already_crawling_list= []

In [64]:
crawling_list = shopping['AREA_NM_PRE'].tolist()

In [66]:
column_lists = [name_list, type_list, address_list, visitor_review_num_list, blog_review_num_list, visitor_review_list, dl_mood_list, dl_topic_list, dl_why_list, dl_20_pop_list]
# ---------------------------------------------------
# ---------------------------------------------------
#-> 처음 돌릴 땐 아래 코드도 실행해줘야 함
for title in crawling_list[0]:
    try:
        driver.switch_to.frame('entryIframe')
    except:
        pass

for title in tqdm(crawling_list):

    no_result = False
    
    driver.switch_to.default_content() # Iframe 빠져나가기
    search_box = driver.find_element_by_css_selector('div.input_box > input')
    search_box.clear()
    search_box.send_keys('제주도' + ' ' + title)
    search_box.send_keys('\n')
    time.sleep(2)

    try:
            # 바로 나온 경우
        driver.switch_to.frame('entryIframe') # Iframe으로 들어가야 함
    except:
        # 바로 나오지 않고 목록에서 선택 해야 하는 경우
        driver.switch_to.frame('searchIframe') # Iframe으로 들어가야 함
        try:
            tmp = driver.find_element_by_css_selector('div.FYvSc').text
            no_result = True
        except:
            try:
                driver.find_element_by_css_selector('li.VLTHu.OW9LQ > div.ouxiq.icT4K > a').click() # 맨 위에 있는 거 click
            except:
                try:
                    driver.find_element_by_css_selector('li.UEzoS.rTjJo > div > a').click()
                except:
                    try:
                        driver.find_element_by_css_selector('li.VLTHu.OW9LQ > div.ouxiq > a').cilck()
                    except:
                        driver.find_element_by_css_selector('li.VLTHu.OW9LQ > div.ouxiq > a').click()
            driver.switch_to.default_content() # Iframe 빠져나가기
            time.sleep(1)
            driver.switch_to.frame('entryIframe') # Iframe으로 들어가야 함

    time.sleep(2)
    if no_result:
        for col in column_lists:
            col.append('no_result')
    else:
        name_list, type_list, address_list, visitor_review_num_list, blog_review_num_list, visitor_review_list, dl_mood_list, dl_topic_list, dl_why_list, dl_20_pop_list = naver_contents(name_list, type_list, address_list, visitor_review_num_list, blog_review_num_list, visitor_review_list, dl_mood_list, dl_topic_list, dl_why_list, dl_20_pop_list, already_crawling_list)
    org_list.append(title)
    print(name_list[-1], visitor_review_num_list[-1], visitor_review_list[-1], dl_20_pop_list[-1])
    driver.switch_to.default_content() # Iframe 빠져나가기

  1%|▋                                                                                 | 1/122 [00:04<08:47,  4.36s/it]

no_result no_result no_result no_result


  2%|█▎                                                                                | 2/122 [00:22<17:08,  8.57s/it]

HAN컬렉션 334 no_result 


  2%|██                                                                                | 3/122 [01:10<40:26, 20.39s/it]

...

함덕민속오일시장 0  


 99%|███████████████████████████████████████████████████████████████████████████████▎| 121/122 [55:27<00:18, 18.24s/it]

현승이네농장 0  


100%|████████████████████████████████████████████████████████████████████████████████| 122/122 [56:04<00:00, 27.57s/it]

시장분식 97 {'음식이 맛있어요': 42, '가성비가 좋아요': 17, '재료가 신선해요': 11, '양이 많아요': 9, '친절해요': 5} 


In [74]:
data_tmp = pd.DataFrame([name_list, type_list, address_list, visitor_review_num_list, blog_review_num_list, visitor_review_list, dl_mood_list, dl_topic_list, dl_why_list, dl_20_pop_list, org_list]).T

In [75]:
data_tmp

,0,1,2,3,4,5,6,7,8,9,10
0,no_result,no_result,no_result,no_result,no_result,no_result,no_result,no_result,no_result,no_result,b일상잡화점
1,HAN컬렉션,종합패션,제주 제주시 노연로 12,334,10,no_result,,,,,han컬렉션
2,갈중이,의류제작,제주 서귀포시 안덕면 사계남로216번길 24-61,49,31,no_result,,,,,갈중이
3,게른,인테리어소품,제주 서귀포시 효돈로 34 게른,21,61,no_result,,,,,게른
4,곱은과일,과일,제주 제주시 동문로 10,417,5,"{'과일이 맛있어요': 65, '신선해요': 48, '친절해요': 39, '제철 과...",,,,,곱은과일
...,...,...,...,...,...,...,...,...,...,...,...
117,한마음한라봉,과일,제주 제주시 애월읍 애월해안로 32,0,0,,,,,,한마음한라봉
118,no_result,no_result,no_result,no_result,no_result,no_result,no_result,no_result,no_result,no_result,한복담
119,함덕민속오일시장,시장,제주 제주시 조천읍 함덕16길 15-13,0,54,,,,,,함덕민속오일시장
120,현승이네농장,과일,제주 제주시 애월읍 신엄리 16,0,1,,,,,,현승이네농장


In [76]:
data_tmp.columns = ['name_list', 'type_list', 'address_list',
       'visitor_review_num_list', 'blog_review_num_list',
       'visitor_review_list', 'dl_mood_list', 'dl_topic_list', 'dl_why_list',
       'dl_20_pop_list', 'org_list']

In [77]:
data_tmp = data_tmp[data_tmp.name_list != "no_result"]

In [79]:
data_tmp.shape

(104, 11)

In [80]:
data_tmp.to_csv('data/쇼핑데이터_네이버크롤링.csv', encoding='utf-8-sig', index=False)